In [1]:
from glob import glob
import re

import ipywidgets as widgets
from IPython.display import display, HTML, Javascript, clear_output

from langchain.document_loaders import DirectoryLoader
from langchain.schema import Document
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig
from peft import PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda110.so...


/opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /opt/conda did not contain libcudart.so as expected! Searching further paths...
  warn(msg)


In [4]:
# print("Loading transformer embeddings...")
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

print("Loading Vicuna tokenizer")
tokenizer = LlamaTokenizer.from_pretrained("eachadea/vicuna-13b-1.1")

print("Loading Vicuna model")
model = LlamaForCausalLM.from_pretrained("eachadea/vicuna-13b-1.1", load_in_8bit=True, device_map="auto")
# model = PeftModel.from_pretrained(model, "kmnis/DocScribe")

Loading Vicuna tokenizer


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading Vicuna model


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
def get_results(question):
    prompt = f"""### You are an AI mental health counselor with substantial training in therapeutic techniques.
    Be sympathetic to the client and maintain a human-like conversation with the client. Adopt motivational interviewing.
    The client should feel like they can count on you to support them through their mental health journey.
    Don't make them feel like you are pushign them away.
    Use your strong understanding of therapeutic techniques, to counsel the client on their struggles.

### Question:
{question}

### Answer:
"""
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(temperature=0.6, top_p=0.95, repetition_penalty=1.15)

    generation_output = model.generate(input_ids=input_ids, generation_config=generation_config,
                                       return_dict_in_generate=True, output_scores=False, max_new_tokens=200)

    for out in generation_output.sequences:
        out = tokenizer.decode(out)
        out = out.split("### Answer:")[1].split("</s>")[0].strip()
        print(out + "\n")

In [19]:
question = """
Hey, I am Joe and am struggling with an eating disorder.
"""
get_results(question)

Hi Joe, it's nice to meet you. I understand that dealing with an eating disorder can be very difficult and challenging. It takes a lot of courage to seek help and support for this issue. Can you tell me more about what you have been experiencing?



In [ ]:
res